# julia itteration solver sparse src

In [18]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate
using SparseArrays
using Arpack
using LinearAlgebra
using BenchmarkTools, IterativeSolvers, MatrixDepot, Random


struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]


struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct FastUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct OffDiagonal{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

function backward_sub!(F::FastUpperTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = A.n : -1 : 1

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = x[col] / A.nzval[idx]

        # Substitute next values involving x[col]
        for i = A.colptr[col] : idx - 1
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end




backward_sub! (generic function with 1 method)

## miscell_fn

In [3]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random


function gen_Ab(n)
    #n = 100
    # Poisson matrix of dimension n^2=10000, pd and sparse
    A = matrixdepot("poisson", n)
    @show typeof(A)
    # dense matrix representation of A
    #Afull = convert(Matrix, A)
    #@show typeof(Afull)
    # sparsity level
    count(!iszero, A) / length(A)
    b = randn(n^2)
    (A,b)
end

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


gen_Ab (generic function with 1 method)

# Custom src

# SOR

In [19]:

#(α×A×X)+(b)  
#f_mul!(-one(T), D, x, b, r) # r <- b- Ax
function f_mul!(α::T, O::DiagonalIndices, x::AbstractVector, b::AbstractVector, r::AbstractVector  ) where {T}
    A = O.matrix
    r[:] = b
    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : A.colptr[col + 1] - 1
            r[A.rowval[j]] += A.nzval[j] * αx 
        end
    end
    r
end

function sum!(z, r, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        z[i] += r[i]
    end
end


function m_sor!(A, D::DiagonalIndices, w)
    for d_idx in D.diag 
        A.nzval[d_idx]  *= (1/w)
    end
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_sor!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        forward_sub!(F, r)# r <- M_sor\r
        sum!(x, r, A) # x <- x +  M_sor/b
        #x[:] = x+r
        
    end
    x
    
end


function k3_sor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    m_sor!(m_sor, D, w)
    D_ms = DiagonalIndices(m_sor)
    itter_sor!(FastLowerTriangular(m_sor ,D_ms), D, x , b, maxiter)
end



k3_sor (generic function with 1 method)

# SSOR

In [20]:
function gamma_sqrt_diag_mul!( D::DiagonalIndices, b::AbstractVector, w ,b_c)
    A = D.matrix
    for idx in D.diag 
        b[A.rowval[idx]] *=  sqrt( b_c * ((2/w) -1) * A.nzval[idx])
    end
end

function sum2!(x,y,z, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        x[i] =y[i]+z[i]
    end
end

function itter_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    #symetric일때도 필요한지 고려 diag정의는 새로필요한거 같음
    
    T = eltype(b)
    r = zeros(A.n)
    y = zeros(A.n)
        
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r_1 <-  γ * D^(1/2) * b- Ay
        gamma_sqrt_diag_mul!(D,r,w,1)
        forward_sub!(F, r) #r_1 <- m_sor\r_1
        gamma_sqrt_diag_mul!(D,r,w,1)
        backward_sub!(U, r)
        sum!(x, r, A)

    end
    x
end



function k3_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    
    itter_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, maxiter)
end


k3_ssor (generic function with 1 method)

# Cheby_SSOR

In [21]:
function mul_sum!(x_next, x, x_pre, r, α, τ, A)
    for i = 1:A.n 
        x_next[i] = (1-α)*x_pre[i] + α*x[i] + (τ*α*r[i])
    end
    x
end

function sum2!(x_next, x, r, A)
    for i = 1:A.n 
        x_next[i] = x[i] + r[i]
    end
    x
end

function eigMm(A::SparseMatrixCSC, ω::Real)

    Dw = sqrt((2/ω-1)) * Diagonal(sqrt.(diag(A)))
    L = (LowerTriangular(A)- (1-1/ω) * Diagonal(A))*(inv(Dw))

    Meig = inv(cholesky(L*L')) * A
    
    λ_max = eigs(Meig; nev=1, ritzvec=false, which=:LM)[1][1]
    λ_min = eigs(Meig; nev=1, ritzvec=false, which=:SM)[1][1]
    real(λ_max), real(λ_min)
end


function itter_CB_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  λ_max, λ_min, max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    
    δ = ((λ_max - λ_min)/4)^2
    τ = 2/(λ_max + λ_min)
    
    #Assign initial parameter
    β  = 2*τ
    α = 1
    b_c = 2/α - 1
    a = (2/τ -1) * b_c
    κ = τ
    
    T = eltype(b)
    r = zeros(A.n)
    x_pre = zeros(A.n)
    x_next = zeros(A.n)
 
    for i = 1 : max_itter 

        x_pre[:] = x 
        x[:] = x_next
        
        f_mul!(-one(T), D, x, b, r) # r_1 <-  γ * D^(1/2) * b- Ay
        gamma_sqrt_diag_mul!(D,r,w,1)
        forward_sub!(F, r) #r_1 <- m_sor\r_1
        gamma_sqrt_diag_mul!(D,r,w,1)
        backward_sub!(U, r)
        
        if i == 1
            sum2!(x_next, x, τ*r, A)
        else
            mul_sum!(x_next, x, x_pre, r, α, τ, A) # x_next <- (1-α)*x_pre + α*x + (τ*α*r[i])
        end
                
        β = 1 / ( (1/τ)  - β*δ ) 
        α = β / τ
        b_c = ( ( 2*κ*(1- α) ) / β) + 1
        a = ((2/τ) -1) + (b_c-1) * ( (1/τ) + (1/κ) -1 )
        κ = β  + ( (1 - α) * κ)
    end

    x

end



function k3_CB_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    λ_max,λ_min = eigMm(A, w)
    
    itter_CB_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, λ_max, λ_min, maxiter)
end


k3_CB_ssor (generic function with 1 method)

# Richardson

In [29]:

function itter_Richardson!(D::DiagonalIndices, x::AbstractVector, w, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        sum!(x, w*r, A) # x <- x +  wr
        
    end
    x
    
end


function k3_RCS(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_Richardson!(D, x , w, b, maxiter)
end



k3_RCS (generic function with 1 method)

# Jacobi

In [31]:
function mul_inv_d!(D, r)
    A = D.matrix
    for idx in D.diag 
        r[A.rowval[idx]] *=  (1 / A.nzval[idx])
    end
end
    

function itter_JCB!(D::DiagonalIndices, x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        mul_inv_d!(D, r)# r <- D^-1*r
        sum!(x, r, A) # x <- x +  M^-1*r
        #x[:] = x+r 
    end
    x
    
end


function k3_JCB(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_JCB!(D, x , b, maxiter)
end



k3_JCB (generic function with 1 method)

# Gauss-Seidal

In [24]:

function m_GS!(A, D::DiagonalIndices)
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_GS!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        forward_sub!(F, r)# r <- M_sor\r
        sum!(x, r, A) # x <- x +  M_sor/b        
    end
    x
    
end


function k3_GS(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    m_GS = copy(A)
    D = DiagonalIndices(A)
    m_GS!(m_GS, D)
    D_GS = DiagonalIndices(m_GS)
    itter_sor!(FastLowerTriangular(m_GS ,D_GS), D, x , b, maxiter)
end



k3_GS (generic function with 1 method)

# Test

In [25]:
using LinearAlgebra
using SparseArrays

function cartesianidx(k::Int, n::Int)
    """
    {s_i} are on a regular 10 by 10 lattice over the two dimensional domain
    S = [1, 10] \times [1, 10].
    output gives a cartesian index (i, j) of s_k, i.e., 
    S = [s01 s11 s21 ... s91
         s02 s12 s22 ... s92
         s03 s13 s23 ... s93
            ... ... 
         s10 s20 s30 ... s100]
    """
    if k % n == 0
        [n; k ÷ n]
    else
        [k % n; k ÷ n + 1]
    end
end

"""
n_i is the number of points neighbouring s_i, i.e., with distance 1from s_i
N[i, j] = n_k, where cartesianindex(k, 10) = (i, j)
"""

function gen_Ab(n)
    #n = 10
    N = 4 * ones(n, n);
    for k in 1:n
        N[1, k] -= 1
        N[k, 1] -= 1
        N[n, k] -= 1
        N[k, n] -= 1
    end

    """
    6.1 A 10 by 10 lattice example (n = 100)
    """
    A = zeros(n^2, n^2)
    for i in 1:n^2
        for j in 1:n^2
            if i == j
                A[i, j] = 0.0001 + N[i]
            elseif norm(cartesianidx(i, n^2)-cartesianidx(j, n^2)) <= 1.0
                A[i, j] = -1.0
            else
                A[i, j] = 0.0
            end
        end
    end

    A = sparse(A)
    b = randn(n^2)
    
    (A,b)
end



gen_Ab (generic function with 1 method)

In [15]:
A,b = gen_Ab(10)

(
  [1  ,   1]  =  2.0001
  [2  ,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [2  ,   2]  =  3.0001
  [3  ,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [3  ,   3]  =  3.0001
  [4  ,   3]  =  -1.0
  [3  ,   4]  =  -1.0
  [4  ,   4]  =  3.0001
  [5  ,   4]  =  -1.0
  [4  ,   5]  =  -1.0
  ⋮
  [96 ,  96]  =  3.0001
  [97 ,  96]  =  -1.0
  [96 ,  97]  =  -1.0
  [97 ,  97]  =  3.0001
  [98 ,  97]  =  -1.0
  [97 ,  98]  =  -1.0
  [98 ,  98]  =  3.0001
  [99 ,  98]  =  -1.0
  [98 ,  99]  =  -1.0
  [99 ,  99]  =  3.0001
  [100,  99]  =  -1.0
  [99 , 100]  =  -1.0
  [100, 100]  =  2.0001, [0.1932438144638102, 0.7935948797879782, -0.6166603890553979, -0.5669832516983522, 0.8274751304764992, -0.024615985567000922, -0.5674057243062406, 0.29376068105844827, -1.4645315085650001, 0.30882307768829564  …  1.1849611550954835, 0.6798365588500925, 1.333745898443076, -0.9012749434236612, 2.454122200318449, -0.4578421730345891, -0.2810682891356796, 0.10396032107323669, -0.9225630073669616, 0.11743688912518445])

In [38]:
sol_1 = sor(A,b , 0.75 ,maxiter=10000)
sol_2 = ssor(A,b , 0.75 ,maxiter=10000)
b_ = copy(b)
sol_4 = k3_CB_ssor(A, b_, 0.75, 10000)
b_ = copy(b)
sol_5 = k3_RCS(A, b_, 1, 10000)
b_ = copy(b)
sol_6 = k3_GS(A, b_, 10000)
b_ = copy(b)
sol_7 = k3_JCB(A, b_,  10000)


100-element Array{Float64,1}:
  0.24118294592717585 
  0.2891461956851342  
 -0.16731032404018273 
 -0.17443350978268868 
  0.21097560303949062 
 -0.020093714015034722
 -0.24664276888899536 
 -0.15245353262259975 
 -0.5044937553905144  
  0.10345332564051744 
  0.4025876743158178  
 -0.13445096618060293 
 -0.5849060447007972  
  ⋮                   
  0.15414340310348054 
  0.3056057452300107  
  1.2000917728645433  
  0.909736654580879   
  0.8493726056934597  
  0.3047202013169939  
  0.966093413701315   
  0.1395344488098726  
 -0.08963394079222707 
 -0.1273769454449534  
 -0.39646995431041443 
 -0.1395095571147592  

In [39]:
sol_5

100-element Array{Float64,1}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN